## SATCEN-01-03-01 Sentinel-2 DVI, composites and BOA reflectances

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [License](#license)

### <a name="objective">Objective 

As a data processor developer, I want to implement, and package an algorithm processing S2 Level1 datasets producing time series data cube of areas with potential illicit crops based on Sentinel 2 data

* DVI Maps
* Coregistered stack of Sentinel-2 (BOA reflectances)
* Color composites (4,3,2), (8a,4,3), (8,4,3), (8a,4,12), (11,8a,4), (8a,11,2), (12,4,3)

### <a name="data">Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

The data used are Sentinel-2 Level-1C products: Top of atmosphere reflectances in fixed cartographic geometry (combined UTM projection and WGS84 ellipsoid). Level-1C images are a set of tiles of 100 sq km, each of which is approximately 500 MB. These products contain applied radiometric and geometric corrections (including orthorectification and spatial registration). 

The spectral bands of Sentinel-2 Level-1C products are:

| S-2 band                | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 8a  | 9   | 10   | 11   | 12   |
|-------------------------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|------|------|------|
| Central wavelength (nm) | 443 | 490 | 560 | 665 | 705 | 740 | 783 | 842 | 865 | 945 | 1375 | 1610 | 2190 |
| Bandwidth (nm)          | 20  | 65  | 35  | 30  | 15  | 15  | 20  | 115 | 20  | 20  | 30   | 90   | 180  |
| Spatial resolution (m)  | 60  | 10  | 10  | 10  | 20  | 20  | 20  | 10  | 20  | 60  | 60   | 20   | 20   |

### <a name="service">Service Definition

In [1]:
service = dict([('title', 'satcen-01-03-01'),
                ('abstract', 'SATCEN-01-03-01 Sentinel-2 DVI, composites and BOA reflectances'),
                ('id', 'ewf-satcen-01-03-01')])

### <a name="parameter">Parameter Definition 

**Resolution**

Target resolution expressed in meters: 10, 20 or 60

In [2]:
resolution = dict([('id', 'resolution'),
                   ('value', '10'),
                   ('title', 'Target resolution'),
                   ('abstract', 'Target resolution (10m, 20m or 60m)')])

**Cloud percentage threshold**

Cloud percentage threshold for analysis rejection. Above the threshold over the AOI, the processing stops

In [3]:
percentage_threshold = dict([('id', 'percentage_threshold'),
                             ('value', '20.0'),
                             ('title', 'Cloud percentage threshold'),
                             ('abstract', 'Cloud percentage threshold')])

**Area of interest WKT**

Area of interest expressed in Well-Known Text

In [4]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON((14.9997721265064 38.0212298555682,15.0339421243074 38.1469295398089,15.076722011609 38.2943830685215,15.1196939590664 38.4418167678655,15.1628416993738 38.5892613166416,15.2061510925436 38.7366798078584,15.2388261265461 38.8477042183055,16.2649601148006 38.8421486728812,16.2478654087227 37.8528295032852,14.9997726359092 37.8594419593273,14.9997721265064 38.0212298555682))'),
            ('title', 'Protected Area wkt'),
            ('abstract', 'Protected Area wkt')])


** Sentinel-2 flag expression**

Flag expression for pixel selection

In [5]:
flag_expr = dict([('id', 'flag_expr'),
                  ('value', '( saturated_l1a_B4 or scl_water )'),
                  ('title', 'Flag expression for pixel exclusion'),
                  ('abstract', 'Flag expression for pixel exclusion (e.g. saturated_l1a_B4 will exclude pixels having the flag saturated_l1a_B4 set)')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Sentinel-2 product identifier

In [6]:
input_identifier = 'S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Input reference**

This is the Sentinel-2 catalogue reference

In [7]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Data path**

This path defines where the data is staged-in. 

In [8]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [9]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys

import dateutil.parser as parser

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import snappy

import gc

import gdalnumeric

import gdal
import osr
import numpy as np

#### Open the Sentinel-2 product

In [10]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL2A.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()



In [11]:
product_date = parser.parse(product.getStartTime().toString()).date()
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)


#### Resample 

In [12]:
if resolution['value'] == '10':
    reference_band = 'B4'

if resolution['value'] == '20':
    reference_band = 'B5'
    
if resolution['value'] == '60':
    reference_band = 'B1' 

In [13]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')
   
parameters = HashMap()
parameters.put('referenceBand', reference_band)

resample = snappy.GPF.createProduct('Resample', parameters, product)

#### Cloud coverage analysis

In [14]:
HashMap = snappy.jpy.get_type('java.util.HashMap')

BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand0 = BandDescriptor()
targetBand0.name = 'cloud_mask'
targetBand0.type = 'uint16'
targetBand0.expression = 'opaque_clouds_60m'

targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
targetBands[0] = targetBand0
 
parameters = HashMap()
parameters.put('targetBands', targetBands)

cloud_mask = snappy.GPF.createProduct('BandMaths', parameters, resample)

In [15]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
cloud_mask_geo = snappy.GPF.createProduct('Subset', parameters, cloud_mask)

mask_geo_output_name = 'AOI_MASK.tif'
snappy.ProductIO.writeProduct(cloud_mask_geo, mask_geo_output_name, 'GeoTIFF-BigTIFF')

cloud_mask_geo = None
gc.collect()

16

In [16]:
raster_file = gdalnumeric.LoadFile(mask_geo_output_name)

pixel_count_cloud_geo = (raster_file == 255).sum()  # for pixel value = 1

cloud_percent =  float(pixel_count_cloud_geo) / float(raster_file.size) * 100.0

os.remove(mask_geo_output_name)
raster_file = None

In [17]:
cloud_percent

4.1278927408999975

In [18]:
if cloud_percent >= float(percentage_threshold['value']):
    
    exit(0)

#### Clouds below threshold

In [19]:
if not flag_expr['value']:
    dvi_expr = '(B9 - B4) * 10000'
else:
    dvi_expr = '! %s ? (B9 - B4) * 10000' % flag_expr['value']
    
dvi_expr = '(B9 - B4) * 10000'

In [20]:
band_names = resample.getBandNames()
list(band_names)[0:12]

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [21]:
target_bands = list(band_names)[0:12] + ['dvi', 'quality_cloud_confidence', 'quality_snow_confidence', 'quality_scene_classification']

In [22]:
BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(target_bands))

for index, band in enumerate(target_bands):
     
    targetBand = BandDescriptor()
    
    if band == 'dvi':
       
        targetBand.expression = dvi_expr   
    
    else:
    
        targetBand.expression = band

    targetBand.name = band
    targetBand.type = 'float32'
    
    targetBands[index]= targetBand
        

parameters = HashMap()
parameters.put('targetBands', targetBands)

result = snappy.GPF.createProduct('BandMaths', parameters, resample)

#### Subset

In [24]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
subset_result = snappy.GPF.createProduct('Subset', parameters, result)


output_name = '%s_DVI_BOA' % input_identifier

snappy.ProductIO.writeProduct(subset_result, output_name, 'GeoTIFF-BigTIFF')

#### Composites

In [ ]:
ds = gdal.Open(output_name + '.tif')

geo_transform = ds.GetGeoTransform()
proj = ds.GetProjection()

srs=osr.SpatialReference(wkt=proj)

options = ['PHOTOMETRIC=RGB', 'PROFILE=GeoTIFF']

In [ ]:
from skimage.exposure import cumulative_distribution
import matplotlib.pylab as plt
import numpy as np

def cdf(im):
 
    c, b = cumulative_distribution(im) 
    # pad the beginning and ending pixels and their CDF values
    c = np.insert(c, 0, [0]*b[0])
    c = np.append(c, [1]*(255-b[-1]))

    return c

def hist_matching(c, c_t, im):
 
    pixels = np.arange(256)
    # find closest pixel-matches corresponding to the CDF of the input image, given the value of the CDF H of   
    # the template image at the corresponding pixels, s.t. c_t = H(pixels) <=> pixels = H-1(c_t)
    new_pixels = np.interp(c, c_t, pixels) 
    im = (np.reshape(new_pixels[im.ravel()], im.shape)).astype(np.uint8)
 
    return im

In [ ]:
# (4,3,2), (8a,4,3), (8,4,3), (8a,4,12), (11,8a,4), (8a,11,2), (12,4,3)

rgb_composites = [['B4', 'B3', 'B2', 1],
                 ['B8A', 'B4', 'B3', 2],
                 ['B8', 'B4', 'B3', 1],
                 ['B8A', 'B4', 'B12', 2],
                 ['B11', 'B8A', 'B4', 2],
                 ['B8A', 'B11', 'B2', 0],
                 ['B12', 'B4', 'B3', 0]]


for rgb_composite in rgb_composites:

    red_radiance  = subset_result.getBand(rgb_composite[0])
    green_radiance = subset_result.getBand(rgb_composite[1])
    blue_radiance = subset_result.getBand(rgb_composite[2])

    w = red_radiance.getRasterWidth()  
    h = red_radiance.getRasterHeight()

    red_radiance_data = np.zeros(w * h, np.float32)
    red_radiance.readPixels(0, 0, w, h, red_radiance_data)
    red_radiance_data.shape = h, w

    green_radiance_data = np.zeros(w * h, np.float32)
    green_radiance.readPixels(0, 0, w, h, green_radiance_data)
    green_radiance_data.shape = h, w

    blue_radiance_data = np.zeros(w * h, np.float32)
    blue_radiance.readPixels(0, 0, w, h, blue_radiance_data)
    blue_radiance_data.shape = h, w


    red = (red_radiance_data*256/(np.amax(red_radiance_data)-np.amin(red_radiance_data)))

    green = (green_radiance_data*256/(np.amax(green_radiance_data)-np.amin(green_radiance_data)))

    blue = (blue_radiance_data*256/(np.amax(blue_radiance_data)-np.amin(blue_radiance_data)))

    
    
    rgb_name = '%s_%s_%s_%s.tif' % (output_name,
                                    rgb_composite[0],
                                    rgb_composite[1],
                                    rgb_composite[2])
    
    dst_ds = gdal.GetDriverByName('GTiff').Create(rgb_name, w, h, 3, gdal.GDT_Byte, options=options)
    
    dst_ds.SetGeoTransform(geo_transform)    # specify coords
 
    dst_ds.SetProjection(srs.ExportToWkt()) # export coords to file
    dst_ds.GetRasterBand(1).WriteArray(red.astype(np.uint8))   # write r-band to the raster
    dst_ds.GetRasterBand(2).WriteArray(green.astype(np.uint8))   # write g-band to the raster
    dst_ds.GetRasterBand(3).WriteArray(blue.astype(np.uint8))   # write b-band to the raster
    dst_ds.FlushCache()                     # write to disk
    dst_ds = None
    
    
    if rgb_composite[3] == 0: 
        
        template = cdf(red.astype(np.uint8))
    
    if rgb_composite[3] == 1:
        
        template = cdf(green.astype(np.uint8))
        
    if rgb_composite[3] == 2:
        
        template = cdf(blue.astype(np.uint8))
    
    
    new_red = hist_matching(cdf(red.astype(np.uint8)), 
                            template,
                            green.astype(np.uint8))
    
    new_green = hist_matching(cdf(green.astype(np.uint8)), 
                              template,
                              green.astype(np.uint8))
    
    new_blue = hist_matching(cdf(blue.astype(np.uint8)), 
                              template,
                              blue.astype(np.uint8))
    
    rgb_name = '%s_%s_%s_%s_new.tif' % (output_name,
                                        rgb_composite[0],
                                        rgb_composite[1],
                                        rgb_composite[2])
    
    dst_ds = gdal.GetDriverByName('GTiff').Create(rgb_name, w, h, 3, gdal.GDT_Byte, options=options)
    
    dst_ds.SetGeoTransform(geo_transform)    # specify coords
 
    dst_ds.SetProjection(srs.ExportToWkt()) # export coords to file
    dst_ds.GetRasterBand(1).WriteArray(red.astype(np.uint8))   # write r-band to the raster
    dst_ds.GetRasterBand(2).WriteArray(new_green)   # write g-band to the raster
    dst_ds.GetRasterBand(3).WriteArray(new_blue)   # write b-band to the raster
    dst_ds.FlushCache()                     # write to disk
    dst_ds = None
    
ds = None

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.